In [ ]:
import os
os.getcwd()
# os.chdir('..')
print(f'Current working directory is {os.getcwd()}')

Current working directory is c:\Users\blond\Documents\NssProjects\sleigh_the_model


In [2]:
import pandas as pd
import scripts.features as ft

In [3]:
file = 'data/dataformodel.csv'

In [4]:
data = ft.load_data(file, 3)

In [5]:
features_to_lag = [
    'pce',
    'inflation',
    'unemployment_rate',
    'revolving_credit',
    'nonrevolving_credit',
    'total_credit'
]

In [7]:
data_with_lag = ft.add_lag_features(data, 3, features_to_lag)